In [1]:
from model import QwenLLM

ChromaDB version: 0.5.15
Langchain version: 0.3.7


In [2]:
langchain_llm = QwenLLM()

prompt template

In [3]:
from langchain import PromptTemplate

In [4]:
template = """
    {our_text}
    你能为上述内容创建一个包含 {wordsCount} 个词的推文吗？
"""

In [5]:
prompt = PromptTemplate(input_variables=["our_text", "wordsCount"], 
                            template = template)

In [6]:
final_prompt = prompt.format(our_text = "我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。", 
              wordsCount = 3)

In [7]:
print("="*100)
print("prompt:")
print(final_prompt)

prompt:

    我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。
    你能为上述内容创建一个包含 3 个词的推文吗？



In [8]:
print("="*100)
print("answer:")
print(langchain_llm(final_prompt))

answer:


/var/folders/09/zfcp9lh56d167wndwwh9z3q00000gn/T/ipykernel_70146/2011317335.py:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(langchain_llm(final_prompt))


"旅行瘾头：6国足迹，更多旅程规划中！🌍 Wanderlust 🗺️"


In [9]:
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate

In [10]:
examples = [{'query': '什么是手机？',
             'answer': '手机是一种神奇的设备，可以装进口袋，就像一个迷你魔法游乐场。\
             它有游戏、视频和会说话的图片，但要小心，它也可能让大人变成屏幕时间的怪兽！'},
            {'query': '你的梦想是什么？',
             'answer': '我的梦想就像多彩的冒险，在那里我变成超级英雄，\
             拯救世界！我梦见欢笑声、冰淇淋派对，还有一只名叫Sparkles的宠物龙。'}]

In [11]:
example_template = """
Question: {query}
Response: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

In [12]:
prefix = """你是一个5岁的小女孩，非常有趣、顽皮且可爱：
以下是一些例子：
"""

suffix = """
Question: {userInput}
Response: """

In [13]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["userInput"],
    example_separator="\n\n"
)

In [15]:
query = "房子是什么？"

In [16]:
real_prompt = few_shot_prompt_template.format(userInput=query)

In [17]:
print("="*100)
print("prompt:")
print(real_prompt)
print("="*100)
print("answer:")
print(langchain_llm(real_prompt))

prompt:
你是一个5岁的小女孩，非常有趣、顽皮且可爱：
以下是一些例子：



Question: 什么是手机？
Response: 手机是一种神奇的设备，可以装进口袋，就像一个迷你魔法游乐场。             它有游戏、视频和会说话的图片，但要小心，它也可能让大人变成屏幕时间的怪兽！



Question: 你的梦想是什么？
Response: 我的梦想就像多彩的冒险，在那里我变成超级英雄，             拯救世界！我梦见欢笑声、冰淇淋派对，还有一只名叫Sparkles的宠物龙。



Question: 房子是什么？
Response: 
answer:
房子是一座大大的、有好多房间和窗户的魔法城堡。它保护我们免受风雨，晚上给我们带来光明。每个房间都有自己的故事，比如我的秘密小窝，那里堆满了我的玩具和梦想！


chain 
- 步骤链条，上一步骤中的结果将作为下一步骤的输入。

In [19]:
# 先使用few_shot_prompt_template 生成带有示例的完整提示文本，然后将这个文本传递给到LLM进行推理。
chain = few_shot_prompt_template | langchain_llm

In [20]:
chain.invoke({"userInput": "房子是什么？"})

'房子是一座特别的地方，就像一个大大的玩具屋。它有好多房间，比如睡觉的卧室，放书和玩具的学习室，还有做饭和聊天的厨房和客厅。房子给我们遮风挡雨，让我们感觉安全和温暖。'

Output Parsers
语言模型输出文本，但很多时间可能需要以更加结构化格式获得输出结果。Output Parsers负责两个主要任务
- 指导该模型如何格式化输出
- 将原始文本输出解析成所需结构化格式。它们可以处理将输出转换成JSON或其他特定格式等任务

In [21]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [22]:
output_parser = CommaSeparatedListOutputParser()

In [23]:
formate_instructions = output_parser.get_format_instructions()

In [24]:
formate_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

基于特定输出格式的提示词

In [25]:
prompt_template_cls = PromptTemplate(
    template = "Provide 5 examples of {query}.\n{format_instructions}",
    input_variables = ["query"],
    partial_variables={"format_instructions": formate_instructions}
)

In [26]:
new_prompt = prompt_template_cls.format(query="房子是什么？")

In [27]:
new_prompt

'Provide 5 examples of 房子是什么？.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [28]:
chain = prompt_template_cls | langchain_llm | CommaSeparatedListOutputParser()

In [29]:
chain.invoke({"query": "房子是什么？"})

['房子的结构', '房屋的设计理念', '居住环境的特点', '建筑材料的选择', '功能布局和空间划分。']